Generalization for obtaining the EER from a classifier, its test set and labels.

Differs from the eer_scorer.py previously done. It does not include dealing with file names or plots.

In [1]:
%run fnmr_vs_fmr.ipynb
%matplotlib inline
import numpy as np
import pdb
import pandas as pd
import os
import csv
import time
import itertools as it
import scipy.io
import h5py
from sklearn.neural_network import MLPClassifier

def eer_error(estimator, X, y, mated_dir = None):

    classes_train = estimator.classes_
    classes_train = classes_train.astype(int)
    classes_test = np.unique(y)
    classes_test = classes_test.astype(int)
    mated = []
    nonmated = []
    
    proba = estimator.predict_proba(X)

    for user_train, user_test in it.product(classes_train, classes_test): 
        proba_user = proba[y == user_test]
        mean_proba = np.mean(proba_user[:, user_train.astype(int)-1])
        
        if mated_dir != None:
            to_write = f'{user_train}; {user_test}; {mean_proba}\n'
            mated_name = '{}mated.txt'.format(mated_dir)
            nonmated_name = '{}nonmated.txt'.format(mated_dir)

        if user_train == user_test: #mated scores 
            if mated_dir != None:
                with open(mated_name, 'at') as mated_file:
                    mated_file.write(to_write)
            mated = np.append(mated, mean_proba)            
        else:
            if mated_dir != None:
                with open(nonmated_name, 'at') as nonmated_file:
                    nonmated_file.write(to_write)
            nonmated = np.append(nonmated, mean_proba)

    if mated.size < 1 or nonmated.size < 1:
        print('mated or non mated are scalars')
        return np.nan
    else:     
        fnmr, fmr, eer, x_eer = fnmr_vs_fmr(np.ravel(mated), np.ravel(nonmated))      
    return -eer